In [150]:
#creating a small test file for convenience
def read(f):
    for l in open(f):
        yield eval(l)
        
x = open("opt_dataset/opt.json")
i = 0
training = open("opt_dataset/small_opt.json",'w')

for l in x:
    if (i>=0) and (i<5000):
        training.write(l)
        if(i==0):
            print l
    i += 1
training.close()

{'userName': 'Roxanne Liu', 'recievedDate': '11/08/2015', 'docID': 'ICEB-2015-0002-12058', 'comment': 'It is very enssential for America to keep the international students. They are epquiped with skills and knowledge. The development of America relies on talents. Competition should not be the reason to ask them to leave.', 'postedDate': '11/11/2015'}



In [151]:
#forming the initial vocabulary on the training set of 600 samples
#Training set: 1-600
#Test set: 601-700
#Not using stemming currently, need to modify later

from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model

comments = []
usernames = []
count = 0
for i in read('opt_dataset/small_opt.json'):
    comments.append(i['comment'])
    usernames.append(i['userName'])


punctuation = set(string.punctuation)
stemmer = PorterStemmer()

# extracting the vocab
from sets import Set
vocab = Set()
for i in range(600):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    for w in r.split():
        vocab.add(w)
        
# extracing all the known labels
labels = []
for l in open('opt_dataset/labels_1-700.txt'):
    if(eval(l) == 1):
        labels.append(1)
    else:
        labels.append(0)

In [152]:
# Multinomial naive bayes!

pos_wordCount_mult = defaultdict(int)
neg_wordCount_mult = defaultdict(int)
for w in vocab:
    pos_wordCount_mult[w] = 0
    neg_wordCount_mult[w] = 0
numberpos_mult = 0
numberneg_mult = 0

# calculating the probabilites
for i in range(600):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    bag = r.split()
    for w in bag:
        if(labels[i] == 1):
            pos_wordCount_mult[w] += 1
            numberpos_mult += 1
        else:
                neg_wordCount_mult[w] += 1
                numberneg_mult += 1
                
pos_wordprob_mult = defaultdict(float)
neg_wordprob_mult = defaultdict(float)

for w in vocab:
    pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
    neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)


In [153]:
import math
def predpos_mult(words):
    sum = 0.0
    for w in words:
        if(w in vocab):
            sum += math.log(pos_wordprob_mult[w])
    return (math.log(posprob) + sum)
            
def predneg_mult(words):
    sum = 0.0
    for w in words:
        if(w in vocab):
            sum += math.log(neg_wordprob_mult[w])
    return (math.log(negprob) + sum)
            
predictions_mult = []
LLP = []
LLN = []
for i in range(len(comments)):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    words = r.split()
    x = predpos_mult(words)
    y = predneg_mult(words)
    LLP.append(x)
    LLN.append(y)
    if(x > y):
        predictions_mult.append(1)
    else:
        predictions_mult.append(0)

In [ ]:
# Semi supervised learning, Some kind of expectation maximization...
# Refer to https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Semi-supervised_parameter_estimation
num_iter = 0
print "Number of comments for semi-supervised estimation:",len(comments)

# changing vocabulary to include all of the comments...
vocab = Set()
for i in range(len(comments)):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    for w in r.split():
        vocab.add(w)
        
# running the iterative process a fixed number of times. 

while(num_iter < 6):
    pos_wordCount_mult = defaultdict(int)
    neg_wordCount_mult = defaultdict(int)
    for w in vocab:
        pos_wordCount_mult[w] = 0
        neg_wordCount_mult[w] = 0
    numberpos_mult = 0
    numberneg_mult = 0
    # calculating the probabilites
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        bag = r.split()
        for w in bag:
            if(predictions_mult[i] == 1):
                pos_wordCount_mult[w] += 1
                numberpos_mult += 1
            else:
                    neg_wordCount_mult[w] += 1
                    numberneg_mult += 1

    pos_wordprob_mult = defaultdict(float)
    neg_wordprob_mult = defaultdict(float)

    for w in vocab:
        pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
        neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)

    predictions_mult = []
    LLP = []
    LLN = []
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        words = r.split()
        x = predpos_mult(words)
        y = predneg_mult(words)
        LLP.append(x)
        LLN.append(y)
        if(x > y):
            predictions_mult.append(1)
        else:
            predictions_mult.append(0)
    
    num_iter += 1
    print "iteration number:",num_iter

Number of comments for semi-supervised estimation: 5000
iteration number: 1
iteration number: 2
iteration number: 3
iteration number: 4
iteration number:

In [155]:
train_accuracy = 0.0
test_accuracy = 0.0
for i in range(700):
    if(predictions_mult[i] == labels[i]):
        if i<600:
            train_accuracy += 1
        else:
            test_accuracy += 1
    elif(i>=600  and i<700):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        words = r.split()
        x = predpos_mult(words) // Likelihood of positive
        y = predneg_mult(words) // Likelihood of negative
        print comments[i], "True label:",labels[i],"Predicted label:",predictions_mult[i],"positive:",x,"negative:",y
print "\n","Test accuracy:",test_accuracy/100,"Train accuracy:", train_accuracy/600 

I see absolutely no reason for expanding the opportunities F-1 nonimmigrant students who merely make it much harder for United States young people who have such a difficult time finding work. Please do not allow additional visas or additional time for F-1 individuals to work in the U.S. True label: 0 Predicted label: 1 positive: -298.372833802 negative: -305.006583436
Enough is Enough. What ever happened to WE THE PEOPLE? True label: 0 Predicted label: 1 positive: -61.6589986102 negative: -62.2897943683
Please allow current non-immigrant students studying STEM or recent graduates to compete for jobs. There are enough talented workers looking for jobs. Please do not allow immigrants to take these jobs away from them. True label: 0 Predicted label: 1 positive: -211.729850267 negative: -212.910692857
As a US citizen, I personally think OPT program abuse is too prevailing to be acceptable for US public. However, I do believe there should be common ground as US should retain talented foreig